# MNIST 数据下载


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import tempfile

import numpy
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

In [2]:
import os
if not os.path.exists('MNIST_data/'):
    os.mkdir('MNIST_data/')
mnist=read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# 使用简单的网络来实现手写体的识别
为了用python实现高效的数值计算，我们通常会使用函数库，比如NumPy，会把类似矩阵乘法这样的复杂运算使用其他外部语言实现。不幸的是，从外部计算切换回Python的每一个操作，仍然是一个很大的开销。如果你用GPU来进行外部计算，这样的开销会更大。用分布式的计算方式，也会花费更多的资源用来传输数据。

TensorFlow也把复杂的计算放在python之外完成，但是为了避免前面说的那些开销，它做了进一步完善。Tensorflow不单独地运行单一的复杂计算，而是让我们可以先用图描述一系列可交互的计算操作，然后全部一起在Python之外运行。

In [4]:
import tensorflow as tf
x_input_data=tf.placeholder(tf.float32,[None,784],name='x_input_data')
x_input_label=tf.placeholder(tf.float32,[None,10],name='x_input_label')
w=tf.Variable(tf.zeros([784,10]))
b=tf.Variable(tf.zeros([10]))
y_out=tf.nn.softmax(tf.matmul(x_input_data,w)+b)
cross_entropy=-tf.reduce_sum(x_input_label*tf.log(y_out))
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
init_op=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    for step in range(1001):
        batch_x_data,batch_x_label=mnist.train.next_batch(100)
        correct_prediction=tf.equal(tf.argmax(y_out,1),tf.argmax(batch_x_label,1))
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,'float'))
        [acc,y_out_train,train_loss,_]=sess.run([accuracy,y_out,cross_entropy,train_step,],
                                            feed_dict={x_input_data:batch_x_data,x_input_label:batch_x_label})
        if step%100==0:
            print("step:{} train loss:{} validation acc:{}".format(step,train_loss,acc))

step:0 train loss:230.258544921875 validation acc:0.12999999523162842
step:100 train loss:35.18122100830078 validation acc:0.8999999761581421
step:200 train loss:41.98802185058594 validation acc:0.9200000166893005
step:300 train loss:38.32001876831055 validation acc:0.9300000071525574
step:400 train loss:21.2425537109375 validation acc:0.949999988079071
step:500 train loss:28.345348358154297 validation acc:0.9300000071525574
step:600 train loss:42.71527099609375 validation acc:0.8999999761581421
step:700 train loss:32.13701248168945 validation acc:0.8899999856948853
step:800 train loss:43.98413848876953 validation acc:0.8999999761581421
step:900 train loss:23.53229522705078 validation acc:0.9300000071525574
step:1000 train loss:56.62244415283203 validation acc:0.800000011920929


# MNIST进阶-构建一个多层卷积网络


# 权重初始化

为了创建这个模型，我们需要创建大量的权重和偏置项。这个模型中的权重在初始化时应该加入少量的噪声来打破对称性以及避免0梯度。由于我们使用的是ReLU神经元，因此比较好的做法是用一个较小的正数来初始化偏置项，以避免神经元节点输出恒为0的问题（dead neurons）。为了不在建立模型的时候反复做初始化操作，我们定义两个函数用于初始化。

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积和池化

TensorFlow在卷积和池化上有很强的灵活性。我们怎么处理边界？步长应该设多大？在这个实例里，我们会一直使用vanilla版本。我们的卷积使用1步长（stride size），0边距（padding size）的模板，保证输出和输入是同一个大小。我们的池化用简单传统的2x2大小的模板做max pooling。为了代码更简洁，我们把这部分抽象成一个函数

In [6]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])
sess = tf.InteractiveSession()
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
          x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06
step 100, training accuracy 0.78
step 200, training accuracy 0.92
step 300, training accuracy 0.9
step 400, training accuracy 0.94
step 500, training accuracy 0.96
step 600, training accuracy 0.98
step 700, training accuracy 0.96
step 800, training accuracy 0.98
step 900, training accuracy 1
step 1000, training accuracy 0.92
step 1100, training accuracy 0.94
step 1200, training accuracy 0.96
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 1
step 1600, training accuracy 1
step 1700, training accuracy 0.96
step 1800, training accuracy 0.98
step 1900, training accuracy 0.92


# 用自己的理解重写一遍
将网络写成类，将训练写成函数


In [3]:
import tensorflow as tf
import numpy as np
def MNIST_NET(object):
    def __init__(self,labelNum,keep_prob):
        self.labelNum=lebelNum
        self.learningRate=learningRate
        self.keep_prob=keep_prob
    def weight_variable(self,shape,name):
        return tf.Variable(tf.truncated_normal(shape=shape,stddev=0.1),name=name)
    def bias_variable(self,shape,anme):
        return tf.constant(0.1,shape=shape,name=name)
        
    def interface(self,inputData):
        #将输入reshape成合适的shape
        x_image=tf.reshape(inputData,shape=[-1,28,28,1])
        
        #第一层卷积层
        W_conv1=self.weight_variable([5,5,1,32],'W_conv1')
        b_conv1=self.bias_variable([32],'b_conv1')
        conv1=tf.nn.bias_add(tf.nn.conv2d(
            x_image,W_conv1,strides=[1,1,1,1],padding='SAME'),b_conv1)
        conv1_relu=tf.nn.relu(conv1,name='conv1_relu')
        conv1_maxpool=tf.nn.max_pool(
            conv1_relu,ksize=[1,2,2,1],strides=[1,2,2,1],
            padding='SAME',name='conv1_maxpool')
        
        #第二层卷积层
        W_conv2=self.weight_variable([5,5,32,64],'W_conv2')
        b_conv2=self.bias_variable([64],'b_conv2')
        conv2=tf.nn.bias_add(tf.nn.conv2d(
            conv1_maxpool,W_conv2,strides=[1,1,1,1],padding='SAME'),b_conv2)
        conv2_relu=tf.nn.relu(conv2,name='conv2_relu')
        conv2_maxpool=tf.nn.max_pool(
            conv2_relu,ksize=[1,2,2,1],strides=[1,2,2,1],
            padding='SAME',name='conv2_maxpool')
        
        #密集连接层(全连接层)
        conv2_maxpool_shape=conv2_maxpool.get_shape().as_list()
        nodesNum=conv2_maxpool_shape[1]*conv2_maxpool_shape[2]*conv2_maxpool_shape[3]
        conv2_maxpool_reshape=tf.reshape(conv2_maxpool,shape=[-1,nodesNum])
        W_fc1=self.weight_variable([nodesNum,1024],'W_fc1')
        b_fc1=self.bias_variable([1024],'b_fc1')
        fc1=tf.nn.relu(tf.matmul(conv2_maxpool_reshape,W_fc1)+b_fc1,name='fc1')
        #防止过拟合，使用dropout
        fc1_drop=tf.nn.dropout(fc1,self.keep_prob,name='fc1_drop')
        
        #输出层(全连接层)
        W_fc2=self.weight_variable([1024,self.labelNum],'W_fc2')
        b_fc2=self.bias_variable([self.labelNum],'b_fc2')
        logits=tf.add(tf.matmul(fc1_drop,W_fc2),b_fc2,name='logits')
        return logits
    
    def getAccuracy(self,logits,inputlabel):
        logits_softmax=tf.nn.softmax(logits)
        logitsEqual=tf.equal(tf.argmax(logits_softmax,1),tf.argmax(inputlabel,1))
        acc=tf.reduce_mean(tf.cast(logitsEqual,'float'),name='accuracy')
        return acc
    
def train():
    labelNum=10
    learningRate=0.01
    keep_prob=1
    mynet=MNIST_NET(labelNum,keep_prob)
    inputData=tf.placeholder('float',shape=[None,28,28,1])
    inputlabel=tf.placeholder('float',shape=[None,labelNum])
    logits=mynet.interface(inputData)
    acc=mynet.getAccuracy(logits,inputlabel)
    cross_entropy=tf.nn.softmax_cross_entropy_with_logits(labels=inputlabel,
                                                         logits=logits,
                                                         name='cross_entropy')
    cross_entropy_mean=tf.reduce_mean(cross_entropy,name='cross_entropy_mean')
    train_optimizer=tf.train.GradientDescentOptimizer(learningRate).minimize(cross_entropy_mean,
                                                                            name='train_optimizer')
    with tf.Session() as sess:
        print('-------start training------')
        tf.run(tf.global_variables_initializer())
        for i in range(2001):
            batch = mnist.train.next_batch(50)
            [logits_val,acc_val,loss_val,_]=sess.run([logits,acc,cross_entropy_mean,train_optimizer],
                                           feed_dict={
                                               inputData=batch[0],
                                               inputlabel=batch[1]
                                           })
            if i%100 == 0:
                print('step:%d  loss:%f  acc:%f'%(step,loss_val,acc_val))
        
        print('-------start validating------')
        acc_val=sess.run([acc],
                         feed_dict={
                             inputData=mnist.test.images,
                             inputlabel=mnist.test.labels
                         })
        
if __name__=='__main__':
    train()

SyntaxError: invalid syntax (<ipython-input-3-cb97af51b565>, line 81)